In [7]:
%%bash
mkdir -p /data/reddylab/Revathy/collabs/Susan/logs 
mkdir -p /data/reddylab/Revathy/collabs/Susan/fastq
cd /data/reddylab/Revathy/collabs/Susan/fastq
export PATH=$PATH:/data/reddylab/Revathy/software/sratoolkit.3.0.0-centos_linux64/bin
sbatch -p all \
    --array=0-5 \
    --mem 16G \
    -o /data/reddylab/Revathy/collabs/Susan/logs/download_sra.%a.out \
    <<'EOF'
#!/bin/bash
sra_ids=(SRR5036390 SRR5036391 SRR5036392 SRR5036393 SRR5036394 SRR5036395 SRR5036396)
sra_id=${sra_ids[${SLURM_ARRAY_TASK_ID}]}
fastq-dump --gzip --split-files --origfmt ${sra_id}
EOF

Submitted batch job 29614515


In [2]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/data/gasperini
wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE120nnn/GSE120861/suppl/GSE120861_grna_groups.pilot.txt.gz; gunzip --force -d GSE120861_grna_groups.pilot.txt.gz; awk '{ print $2 "\t" $1}' GSE120861_grna_groups.pilot.txt > raw_gasperini_pilot_guide_sequences.tsv
head -n 10 raw_gasperini_pilot_guide_sequences.tsv 

AGCCTAACGATCGGACCGAG	scrambled_1
TGTCCAGTTCCGTAGGATGG	scrambled_2
TATTCGTACCGGGCAGCAGG	scrambled_3
CGGTGGGGCCGCTCACTAGG	scrambled_4
CTGAACTTCGGTAAAGGCGG	scrambled_5
CTCACCGTAGAGACGGCACG	scrambled_6
ACCGCTGCCCTAACCACTGG	scrambled_7
GCTGTATATCGGCGCCCCGG	scrambled_8
ATGCGGCGAACTGCCGTAAG	scrambled_9
ACTGCCTCGCGATTGACTGG	scrambled_10


--2023-11-06 09:59:41--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE120nnn/GSE120861/suppl/GSE120861_grna_groups.pilot.txt.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.7, 2607:f220:41e:250::7, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38546 (38K) [application/x-gzip]
Saving to: ‘GSE120861_grna_groups.pilot.txt.gz’

     0K .......... .......... .......... .......              100% 2.51M=0.01s

2023-11-06 09:59:42 (2.51 MB/s) - ‘GSE120861_grna_groups.pilot.txt.gz’ saved [38546/38546]



### create symlink for data

Aim to have all fastq in the same directory for processing it through cellranger count. Now all of the samples in multiple different directories separated by SRRs and samples

data for low_moi pilot screen: `/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/low_moi`

In [31]:
import os
import ntpath

path = "/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale"

#recursively walk through the directory to find folders
for root, dir, files in os.walk(path):
    for file in files:
        dirname = ntpath.basename(root)
        ori = root + '/' + file
        dest =  root + '/' + dirname + file
        os.rename(ori, dest)


In [1]:
%%bash
mkdir -p /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/bam
cd /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi
find . -mindepth 2 -name "*.bam" -exec mv {} ../../bam \;

In [7]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale
find . -mindepth 4 -type f -exec mv {} . \;

In [34]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale
mv at_scale_screen*/*MissingLibrary*/* /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale

In [35]:
%%bash
cd /data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/at_scale
rm -r at_scale_screen*

In [12]:
import os
from os import path
import re

os.chdir('/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi')
path_to_directory = "/data/gersbachlab/Revathy/collabs/Susan/data/gasperini/fastq/high_moi"
file_num = 0
for f in os.listdir(path_to_directory):
    res = re.findall("_MissingLibrary_1", f)[0]
    s1 = f.split(res)[0]
    s2 = f.split(res)[-1]
    s3 = s1+s2
    os.rename(f,s3)
      